## this is for testing usage only

In [46]:
import requests

import pandas as pd

from bs4 import BeautifulSoup

from datetime import datetime

In [47]:
#import news data and season duration
df_season = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/season_duration.csv')
df_nba_news_archive = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/nba_news_archive.csv')
df_nba_news_archive_600 = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/nba_news_archive_600.csv')
df_nba_news_archive_600_1200 = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/nba_news_archive_600_1200.csv')
df_nba_news_archive_1200_1640 = pd.read_csv('/Users/annabelitong/desktop/annabel/uni/ds105l/group_project/DS105L-Project/Data/nba_news_archive_1200_1640.csv')


In [48]:
news_0 = df_nba_news_archive.copy()
news_600 = df_nba_news_archive_600.copy()
news_600_1200 = df_nba_news_archive_600_1200.copy()
news_1200_1600 = df_nba_news_archive_1200_1640.copy()

In [49]:
from tqdm import tqdm

def find_author(df_news):
    list_url = list(df_news["url"])
    author_list = []
    for url in tqdm(list_url, desc="Finding authors"):
        try:
            r = requests.get(url) 
            soup = BeautifulSoup(r.content, "html.parser") 
            author = soup.find("span", attrs ={"class": "entry-header__author"}).contents[0]
            author_list.append(author)
        except:
            author = "NA"
            author_list.append(author)
    df_news.loc[:, "author"] = author_list
    return df_news


In [50]:
import pandas as pd
import asyncio
import aiohttp
from bs4 import BeautifulSoup


async def get_html(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            return await response.text()


async def get_author(html):
    soup = BeautifulSoup(html, "html.parser")
    author = soup.find("span", {"class": "c-byline__item"})
    if author is not None:
        return author.get_text().strip()
    else:
        return "N/A"


async def find_authors(url_list):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for url in url_list:
            tasks.append(get_html(url))
        html_list = await asyncio.gather(*tasks)
    tasks = []
    for html in html_list:
        tasks.append(get_author(html))
    author_list = await asyncio.gather(*tasks)
    return author_list


# Load the news dataset
# Keep the first 500 rows of data for testing purposes
df_news = news_0[:500]

# Create a list of URLs
url_list = df_news.url.tolist()

# Run the asynchronous function to get the authors
loop = asyncio.get_event_loop()
author_list = loop.run_until_complete(find_authors(url_list))
loop.close()

# Add the authors to the dataframe
df_news['author'] = author_list


RuntimeError: This event loop is already running

In [ ]:
df_news

,Unnamed: 0,title,time,url
0,0,"Bucks’ George Hill Returns To Milwaukee, Encou...","October 24, 2020",http://global.nba.com/news/bucks-george-hill-r...
1,1,"NBA, WNBA Ramp Up Efforts To Get Out The Vote ...","October 24, 2020",http://global.nba.com/news/nba-wnba-ramp-up-ef...
2,2,"Pat Riley Discusses Heat Future, Plans For Kee...","October 23, 2020",http://global.nba.com/news/pat-riley-discusses...
3,3,Report: Amar’e Stoudemire Joins Steve Nash’s C...,"October 23, 2020",http://global.nba.com/news/report-amare-stoude...
4,4,ESPN To Host Virtual 2020 NBA Draft Presented ...,"October 22, 2020",http://global.nba.com/news/espn-to-host-virtua...
...,...,...,...,...
495,495,McCollum’s Clutch Performance Gives Blazers Co...,"August 15, 2020",http://global.nba.com/news/mccollums-clutch-pe...
496,496,"Blazers Make Playoffs, Oust Grizzlies With 126...","August 15, 2020",http://global.nba.com/news/blazers-make-playof...
497,497,Jusuf Nurkic’s Grandmother Has Died From COVID-19,"August 15, 2020",http://global.nba.com/news/jusuf-nurkics-grand...
498,498,Portland’s Lillard Named KIA NBA Player Of The...,"August 15, 2020",http://global.nba.com/news/portlands-lillard-n...


In [ ]:
#news_0_author = find_author(news_0[])

In [ ]:
author_list

NameError: name 'author_list' is not defined

In [ ]:
news_0[87:88]["url"]

87    http://global.nba.com/news/report-bam-adebayo-...
Name: url, dtype: object

In [ ]:
credit_article = []
credit_followers = []
credit_tweets = []
for names in list(news_0_author["author"]):
    if names == "NA":
        credit_article.append(names)
        credit_followers.append(names)
        credit_tweets.append(names)
    elif "NBA.com" in names or "NBA News" in names or "Press" in names or "Official Release" in names or "NBA Twitter" in names:
        credit_article.append("Default")
        credit_followers.append("Default")
        credit_tweets.append("Default")
    else:
        try:
            player_search = names.replace(" ", "-")
            url = "https://muckrack.com/" + player_search
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            html = soup.find_all('a', attrs={'class':"profile-edit-btn"})
            if len(html) == 0:
                num_articles = "none"
                credit_article.append(num_articles)
            else:
            
                for line in html:
                        if "See all" in line.text:
                            span_elem = line.find('span')
                            text_content = span_elem.text.strip()
                            num_articles = text_content.split()[0].replace(',', '')
                            credit_article.append(num_articles)
                            break

            tweet = soup.find_all('p', attrs={'class':"fs-6 mt-0 mb-6"})
            tweet_num = tweet.get_all("strong")
            followers = tweet_num[0].strong.text.replace(',', '')
            tweets = tweet_num[1].strong.text.replace(',', '')

        except:
            if len(credit_followers) < len(credit_article):
                credit_followers.append("None")
            if len(credit_tweets) < len(credit_article):
                credit_tweets.append("None")

            
            
            

In [ ]:
credit_article

['Default',
 'Default',
 'none',
 '618',
 '199',
 'Default',
 'Default',
 'NA',
 'none',
 'Default',
 'Default',
 'Default',
 'Default',
 'NA',
 '618',
 'Default',
 'none',
 'Default',
 'none',
 'Default']

In [ ]:
url = "https://twitter.com/search?q=Steve%20Aschburner&src=typed_query&f=user"
r = requests.get(url)
soup =BeautifulSoup(r.text, 'html.parser')
html = soup.find('div', attrs={'class':"css-1dbjc4n r-1wbh5a2 r-dnmrzs r-1ny4l3l"})

In [ ]:
html